In [1]:
import glob
import itertools
import pandas as pd

In [2]:
MAPPING = (
    ('Barschaft (teilweise eigene Berechnung)', 'barschaft'),
    ('Gulden Inventarsumme', 'invenatar_summe_norm_fl'),
    ('Kreuzer Inventarsumme', 'invenatar_summe_norm_kr'),
    ('Gulden vor Abzug Passiva', 'vor_passiva_fl'),
    ('Kreuzer vor Abzug Passiva', 'vor_passiva_kr'),
    ('Gulden nach Abzug Passiva', 'nach_passiva_fl'),
    ('Kreuzer nach Abzug Passiva', 'nach_passiva_kr'),
    ('Buch/Bücher', 'buecher'),    
)

In [3]:
orig_files = glob.glob('../rita_data_ibk/orig_files/*.xlsx')[1:]

In [4]:
dfs = []
for file in orig_files:
    keys = list(pd.read_excel(file, None).keys())
    for x in keys:
        df = pd.read_excel(file, x)
        dfs.append(df)

In [5]:
inst, _ = Institution.objects.get_or_create(written_name="Südtiroler Landesarchiv")

In [6]:
def create_entry(row, repo_name='Südtiroler Landesarchiv'):
    signatur = row['Signatur']
    vb = signatur.split(',')[0]
    year = vb[-4:]
    vb_sig = "{}, {}".format(repo_name, vb)
    vb_obj, _ = VerfachBuch.objects.get_or_create(signatur=vb_sig)
    inv, _ = InventoryEntry.objects.get_or_create(inv_signatur="{}, {}".format(repo_name, signatur))
    inv.is_located_in = vb_obj
    inv.save()
    return inv
    

In [7]:
def relate_person(row, rel_type, persons, entry):
    for y in persons:
        if y.startswith(')'):
            pers = None
        elif y.startswith('[Vorstehe'):
            pers = None
        elif y.startswith('Im Rahm'):
            pers = None
        elif y.startswith('[die Personen'):
            pers = None
        elif y.startswith('n.a'):
            pers = None
        else:
            try:
                pers, _ = Person.objects.get_or_create(written_name=y)
            except Exception as e:
                pers = None
        if pers:
            getattr(entry, rel_type, None).add(pers)
    return entry
            
       

In [8]:
def add_more_data(row, entry, mapping):
    for item in mapping:
        setattr(entry, item[1], row[item[0]])
        try:
            entry.save()
        except Exception as e:
            setattr(entry, item[1], None)
    return entry

In [9]:
for x in dfs:
    for i, row in x.iterrows():
        entry = create_entry(row)
        if row['Buch/Bücher'].startswith('j'):
            entry.buecher_sys = 'Bücher'
        entry.table_row = row.to_json(force_ascii=False)
        persons = str(row['Beteiligte Personen (Beamte, Gerichtsverpflichtete, Zeugen, ...)']).split('\n')
        relate_person(row, 'adm_person', persons, entry)
        persons = str(row['Beteiligte Personen (Erbsinteressenten, Gerhaben, Anweiser, Verkäufer, Verpächter, Käufer, Pächter, ...)']).split('\n')
        relate_person(row, 'related_person', persons, entry)
        persons = str(row['Genannte Personen']).split('\n')
        relate_person(row, 'other_person', persons, entry)
        entry = add_more_data(row, entry, MAPPING)
        entry.save()

In [ ]:
InventoryEntry.objects.all().delete()